In [1]:
import pandas as pd
import numpy as np

Wine Quality Model
http://www3.dsi.uminho.pt/pcortez/wine/

In [2]:
pip install ucimlrepo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [4]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# data (as pandas dataframes) 
X = wine_quality.data.features 
y = wine_quality.data.targets 

In [5]:
data = pd.concat([X, y], axis = 1)
data.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
unseen_data = data.sample(frac=.20, random_state=42)
df = data.drop(unseen_data.index)

In [7]:
# does my row count match
#data.shape[0], unseen_data.shape[0], df.shape[0]
data.shape[0] == unseen_data.shape[0] + df.shape[0]

True

In [8]:
unseen_data.to_csv('./Data/unseen_WineQuality.csv', index=False) # unseen = testing set
df.to_csv('./Data/WineQuality_tt.csv', index=False)  # tt = training set

In [9]:
data = pd.read_csv('./data/WineQuality_tt.csv')
data.head(1)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [10]:
from pycaret.classification import ClassificationExperiment
s = ClassificationExperiment()

In [11]:
s.setup(data, target='quality', session_id=42)

,Description,Value
0,Session id,42
1,Target,quality
2,Target type,Multiclass
3,Target mapping,"3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5, 9: 6"
4,Original data shape,"(5198, 12)"
5,Transformed data shape,"(5198, 12)"
6,Transformed train set shape,"(3638, 12)"
7,Transformed test set shape,"(1560, 12)"
8,Numeric features,11
9,Preprocess,True


In [12]:
%time
best = s.compare_models()

CPU times: total: 0 ns
Wall time: 0 ns


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.6449,0.0000,0.6449,0.6430,0.6298,0.4450,0.4506,0.0990
et,Extra Trees Classifier,0.6443,0.0000,0.6443,0.6396,0.6293,0.4451,0.4501,0.0750
lightgbm,Light Gradient Boosting Machine,0.6341,0.0000,0.6341,0.6347,0.6223,0.4348,0.4377,0.5300
catboost,CatBoost Classifier,0.6339,0.0000,0.6339,0.6332,0.6222,0.4342,0.4372,4.1120
gbc,Gradient Boosting Classifier,0.5734,0.0000,0.5734,0.5695,0.5580,0.3344,0.3396,0.5660
dt,Decision Tree Classifier,0.5470,0.0000,0.5470,0.5491,0.5459,0.3295,0.3305,0.0100
lda,Linear Discriminant Analysis,0.5445,0.0000,0.5445,0.5107,0.5203,0.2811,0.2868,0.0080
ridge,Ridge Classifier,0.5357,0.0000,0.5357,0.4920,0.4704,0.2341,0.2523,0.0070
lr,Logistic Regression,0.5324,0.0000,0.5324,0.4912,0.4925,0.2429,0.2540,0.8530
qda,Quadratic Discriminant Analysis,0.4805,0.0000,0.4805,0.4920,0.4774,0.2463,0.2502,0.0100


In [13]:
s.models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [14]:
lightgbm = s.create_model('lightgbm')
lightgbm

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6319,0.0000,0.6319,0.6126,0.6172,0.4320,0.4343
1,0.6758,0.0000,0.6758,0.6760,0.6669,0.5000,0.5029
2,0.6209,0.0000,0.6209,0.6343,0.6080,0.4123,0.4159
3,0.5962,0.0000,0.5962,0.6147,0.5797,0.3742,0.3761
4,0.6291,0.0000,0.6291,0.6248,0.6139,0.4260,0.4283
5,0.6374,0.0000,0.6374,0.6378,0.6263,0.4367,0.4403
6,0.6071,0.0000,0.6071,0.5976,0.5970,0.3931,0.3958
7,0.6676,0.0000,0.6676,0.6655,0.6583,0.4911,0.4950
8,0.6391,0.0000,0.6391,0.6394,0.6307,0.4477,0.4492


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [15]:
tuned_lightgbm= s.tune_model(lightgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6071,0.0000,0.6071,0.5747,0.5890,0.3941,0.3967
1,0.6566,0.0000,0.6566,0.6566,0.6463,0.4704,0.4729
2,0.6016,0.0000,0.6016,0.6141,0.5904,0.3862,0.3888
3,0.5687,0.0000,0.5687,0.5439,0.5488,0.3266,0.3294
4,0.6209,0.0000,0.6209,0.6079,0.6051,0.4136,0.4158
5,0.6099,0.0000,0.6099,0.6184,0.5976,0.3931,0.3962
6,0.5962,0.0000,0.5962,0.5911,0.5846,0.3727,0.3763
7,0.6071,0.0000,0.6071,0.5910,0.5952,0.3998,0.4016
8,0.5978,0.0000,0.5978,0.5879,0.5849,0.3812,0.3829


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [16]:
final_lightgbm= s.finalize_model(tuned_lightgbm)

In [17]:
final_lightgbm

Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['fixed_acidity',
                                             'volatile_acidity', 'citric_acid',
                                             'residual_sugar', 'chlorides',
                                             'free_sulfur_dioxide',
                                             'total_sulfur_dioxide', 'density',
                                             'pH', 's...
                 LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, importance_type='split',
                                learning_rate=0.1, max_depth=-1,
                                min_child_samples=20, min_child_weight=0.001,
                                min_split_gain=0.0, n_estimators=100, n_jobs=-1,
                                num_leaves=31, objective=None, random_state=42,
                                reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
                                subsample_for_bin=200000, subsample_freq=0))],
         verbose=False)

In [18]:
s.evaluate_model(final_lightgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [19]:
df = s.predict_model(final_lightgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9641,0.9975,0.9641,0.9642,0.9641,0.9466,0.9466


In [20]:
df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,prediction_label,prediction_score
1950,7.4,0.200,0.35,2.1,0.038,30.0,116.0,0.99490,3.49,0.77,10.3,7,7,0.5817
258,7.0,0.685,0.00,1.9,0.067,40.0,63.0,0.99790,3.60,0.81,9.9,5,5,0.8787
2398,7.1,0.190,0.28,3.6,0.033,16.0,78.0,0.99300,2.91,0.78,11.4,6,6,0.7670
3349,6.6,0.360,0.52,10.1,0.050,29.0,140.0,0.99628,3.07,0.40,9.4,5,5,0.8534
2515,6.2,0.430,0.49,6.4,0.045,12.0,115.0,0.99630,3.27,0.57,9.0,4,4,0.8591


In [21]:
df[df['quality'] != df['prediction_label']]

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,prediction_label,prediction_score
1841,6.4,0.310,0.39,7.50,0.040,57.0,213.0,0.99475,3.32,0.43,10.00,5,6,0.6480
1969,8.8,0.360,0.44,1.90,0.040,9.0,121.0,0.99530,3.19,0.48,9.90,6,5,0.5228
1852,6.4,0.160,0.31,5.30,0.043,42.0,157.0,0.99455,3.35,0.47,10.50,5,6,0.5451
2509,7.1,0.360,0.24,1.80,0.025,32.0,102.0,0.99030,3.34,0.59,12.80,6,7,0.4679
783,6.5,0.400,0.10,2.00,0.076,30.0,47.0,0.99554,3.36,0.48,9.40,6,5,0.6166
3126,8.3,0.270,0.34,10.20,0.048,50.0,118.0,0.99716,3.18,0.51,10.30,5,6,0.6407
5135,6.4,0.230,0.35,10.30,0.042,54.0,140.0,0.99670,3.23,0.47,9.20,5,6,0.5532
1697,6.8,0.280,0.17,13.90,0.047,49.0,162.0,0.99830,3.21,0.51,9.00,6,5,0.5686
4136,6.6,0.190,0.35,1.50,0.037,37.0,107.0,0.99006,3.18,0.68,12.00,7,6,0.5911
1987,6.6,0.200,0.38,7.90,0.052,30.0,145.0,0.99470,3.32,0.56,11.00,7,6,0.5429


In [22]:
data = pd.read_csv('./data/WineQuality_tt.csv')

In [23]:
data.head(1)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [24]:
from pycaret.classification import setup, models, create_model, tune_model, finalize_model, save_model, evaluate_model
cls = setup(data, target='quality', session_id=42)

,Description,Value
0,Session id,42
1,Target,quality
2,Target type,Multiclass
3,Target mapping,"3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5, 9: 6"
4,Original data shape,"(5198, 12)"
5,Transformed data shape,"(5198, 12)"
6,Transformed train set shape,"(3638, 12)"
7,Transformed test set shape,"(1560, 12)"
8,Numeric features,11
9,Preprocess,True


In [25]:
lightgbm=create_model('lightgbm')
lightgbm

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6319,0.0000,0.6319,0.6126,0.6172,0.4320,0.4343
1,0.6758,0.0000,0.6758,0.6760,0.6669,0.5000,0.5029
2,0.6209,0.0000,0.6209,0.6343,0.6080,0.4123,0.4159
3,0.5962,0.0000,0.5962,0.6147,0.5797,0.3742,0.3761
4,0.6291,0.0000,0.6291,0.6248,0.6139,0.4260,0.4283
5,0.6374,0.0000,0.6374,0.6378,0.6263,0.4367,0.4403
6,0.6071,0.0000,0.6071,0.5976,0.5970,0.3931,0.3958
7,0.6676,0.0000,0.6676,0.6655,0.6583,0.4911,0.4950
8,0.6391,0.0000,0.6391,0.6394,0.6307,0.4477,0.4492


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [26]:
tuned_lightgbm= tune_model(lightgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6071,0.0000,0.6071,0.5747,0.5890,0.3941,0.3967
1,0.6566,0.0000,0.6566,0.6566,0.6463,0.4704,0.4729
2,0.6016,0.0000,0.6016,0.6141,0.5904,0.3862,0.3888
3,0.5687,0.0000,0.5687,0.5439,0.5488,0.3266,0.3294
4,0.6209,0.0000,0.6209,0.6079,0.6051,0.4136,0.4158
5,0.6099,0.0000,0.6099,0.6184,0.5976,0.3931,0.3962
6,0.5962,0.0000,0.5962,0.5911,0.5846,0.3727,0.3763
7,0.6071,0.0000,0.6071,0.5910,0.5952,0.3998,0.4016
8,0.5978,0.0000,0.5978,0.5879,0.5849,0.3812,0.3829


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [27]:
final_lightgbm= finalize_model(tuned_lightgbm)

In [28]:
save_model(final_lightgbm, './models/quality_unseen')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['fixed_acidity',
                                              'volatile_acidity', 'citric_acid',
                                              'residual_sugar', 'chlorides',
                                              'free_sulfur_dioxide',
                                              'total_sulfur_dioxide', 'density',
                                              'pH', 's...
                  LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                 colsample_bytree=1.0, importance_type='split',
                                 learning_rate=0.1, max_depth=-1,
                               

In [29]:
import joblib

from sklearn.metrics import classification_report
test = pd.read_csv('./data/unseen_WineQuality.csv')

In [30]:
test.shape

(1299, 12)

In [31]:
cls = joblib.load('./models/quality_unseen.pkl')
cls

Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['fixed_acidity',
                                             'volatile_acidity', 'citric_acid',
                                             'residual_sugar', 'chlorides',
                                             'free_sulfur_dioxide',
                                             'total_sulfur_dioxide', 'density',
                                             'pH', 's...
                 LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, importance_type='split',
                                learning_rate=0.1, max_depth=-1,
                                min_child_samples=20, min_child_weight=0.001,
                                min_split_gain=0.0, n_estimators=100, n_jobs=-1,
                                num_leaves=31, objective=None, random_state=42,
                                reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
                                subsample_for_bin=200000, subsample_freq=0))],
         verbose=False)

In [32]:
unseen_X = test.drop('quality', axis=1)
y = test['quality']

In [33]:
y_pred = cls.predict(unseen_X)
y_pred

0       6
1       5
2       7
3       5
4       5
       ..
1294    6
1295    5
1296    6
1297    6
1298    5
Name: quality, Length: 1299, dtype: int8

In [44]:
print
clr = classification_report(y, y_pred)
print(clr)

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         6
           4       0.38      0.19      0.25        43
           5       0.65      0.67      0.66       402
           6       0.65      0.72      0.68       597
           7       0.66      0.54      0.59       214
           8       0.70      0.39      0.50        36
           9       0.00      0.00      0.00         1

    accuracy                           0.65      1299
   macro avg       0.43      0.36      0.38      1299
weighted avg       0.64      0.65      0.64      1299



# Explanation: 
Here's what each metric in the report means:

- Precision is the ratio of correctly predicted positive observations to the total predicted positives. 
It indicates the accuracy of positive predictions. Precision for each class is calculated as the number of true positives divided by the sum of true positives and false positives.

- Recall (sensitivity or true positive rate) is the ratio of correctly predicted positive observations to the all observations in actual class. It measures the completeness or sensitivity of the model. Recall for each class is calculated as the number of true positives divided by the sum of true positives and false negatives.
  
- F1-score is the weighted average of Precision and Recall. It takes both false positives and false negatives into account. It's a good way to show that a classifer has a good value for both recall and precision. F1-score for each class is calculated as 2 * (precision * recall) / (precision + recall).

- Support is the number of actual occurrences of the class in the specified dataset. It's the number of actual positive samples for each class.

- Accuracy is the ratio of correctly predicted observations to the total observations. It measures the overall correctness of the model.
Accuracy is calculated as the ratio of the number of correct predictions to the total number of predictions.

- Macro avg: Macro average calculates the average metric (precision, recall, F1-score) across all classes. It gives equal weight to each class.

- Weighted average calculates the average metric (precision, recall, F1-score) across all classes, weighted by the number of instances of each class. It's useful when there's class imbalance.

From the output of the classification ran:

#### Each row represents a different class.
#### For class 5, the precision is 0.65, recall is 0.67, and F1-score is 0.66.
#### The accuracy of the model on the entire dataset is 0.65.
#### Macro average and weighted average provide the average metrics across all classes.

In [39]:
[y], [y_pred]

([0       8
  1       5
  2       7
  3       6
  4       6
         ..
  1294    6
  1295    5
  1296    5
  1297    7
  1298    6
  Name: quality, Length: 1299, dtype: int64],
 [0       6
  1       5
  2       7
  3       5
  4       5
         ..
  1294    6
  1295    5
  1296    6
  1297    6
  1298    5
  Name: quality, Length: 1299, dtype: int8])